In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [3]:
import sys
sys.path.append('/home/gd1279/projects/Rainbow')

In [4]:
from __future__ import division
import argparse
import os
from datetime import datetime
import atari_py
import numpy as np
import torch
import wandb

from agent import Agent
from env import make_env
from memory import ReplayMemory
from masker import ALL_MASKERS
from tqdm import tqdm, trange
import pickle
import bz2

import matplotlib
import matplotlib.pyplot as plt
import cv2
import glob
import copy
import imageio
from profilehooks import profile
from guppy import hpy
import time

def timeit(method):
    def timed(*args, **kw):
        start_time = time.time()
        result = method(*args, **kw)
        end_time = time.time()

        print(f'{method.__name__} took {end_time - start_time:.2f} s')

        return result
    return timed

In [5]:
!wandb login 9676e3cc95066e4865586082971f2653245f09b4

wandb: Appending key for api.wandb.ai to your netrc file: /home/gd1279/.netrc
Successfully logged in to Weights & Biases!


In [6]:
api = wandb.Api()

In [20]:
run = api.run('augmented-frostbite/masks-and-pixels/runs/jszl27e5')
hist = run.history()
last_updates = [datetime.fromtimestamp(ts) for ts in hist.tail(10)['_timestamp']]

In [22]:
[(last_updates[k + 1] - last_updates[k]).seconds / 60 for k in range(len(last_updates) - 1)]

[23.1,
 32.68333333333333,
 23.3,
 32.35,
 22.766666666666666,
 33.2,
 23.233333333333334,
 30.816666666666666,
 23.1]

[datetime.datetime(2019, 11, 26, 13, 52, 42, 82798),
 datetime.datetime(2019, 11, 26, 14, 15, 48, 235623),
 datetime.datetime(2019, 11, 26, 14, 48, 29, 978845),
 datetime.datetime(2019, 11, 26, 15, 11, 48, 863929),
 datetime.datetime(2019, 11, 26, 15, 44, 10, 505346),
 datetime.datetime(2019, 11, 26, 16, 6, 57, 60492),
 datetime.datetime(2019, 11, 26, 16, 40, 9, 215315),
 datetime.datetime(2019, 11, 26, 17, 3, 23, 966742),
 datetime.datetime(2019, 11, 26, 17, 34, 13, 207468),
 datetime.datetime(2019, 11, 26, 17, 57, 20, 174044)]